In [1]:
#importing the packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [2]:
# Reading the data from the train.csv file

train_data = pd.read_csv("train.csv", index_col=None, header=0, engine='python' )

In [3]:
# displaying the data using head() function

train_data.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos


In [4]:
# Checking the description , datatypes and null values of the dataset

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2914 entries, 0 to 2913
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Tweet            2914 non-null   object
 1   Target           2914 non-null   object
 2   Stance           2914 non-null   object
 3   Opinion Towards  2914 non-null   object
 4   Sentiment        2914 non-null   object
dtypes: object(5)
memory usage: 114.0+ KB


In [5]:
# Checking all the nan values

train_data.isna().sum()

Tweet              0
Target             0
Stance             0
Opinion Towards    0
Sentiment          0
dtype: int64

In [6]:
## Combining the two columns of "Tweet" and "Target"

train_data["Tweet"] = train_data["Tweet"] + " " + train_data["Target"]

In [7]:
## displaying the 1st row after concatenation

train_data['Tweet'][0]

'@tedcruz And, #HandOverTheServer she wiped clean + 30k deleted emails, explains dereliction of duty/lies re #Benghazi,etc #tcot Hillary Clinton'

In [8]:
train_data['Stance'].value_counts()

AGAINST    1395
NONE        766
FAVOR       753
Name: Stance, dtype: int64

In [9]:
# Performing Label Encoding

#train_data["Stance"] = train_data["Stance"].cat.codes

encode = {
    "AGAINST" : 0,
    "FAVOR"  : 1,       
    "NONE" : 2,    
}

train_data["Stance"] = train_data["Stance"].apply(lambda x: encode[x])
train_data.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,0,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,1,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,0,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,0,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,2,3. The tweet is not explicitly expressing opi...,pos


In [10]:
# Displaying the label encoder values

train_data['Stance']

0       0
1       1
2       0
3       0
4       2
       ..
2909    0
2910    0
2911    0
2912    0
2913    0
Name: Stance, Length: 2914, dtype: int64

In [ ]:
#Installing spacy library

In [12]:
!pip install -U spacy

     |████████████████████████████████| 10.0MB 4.3MB/s 
     |████████████████████████████████| 2.1MB 30.3MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [13]:
! pip install -U spacy-Lookups-data

     |████████████████████████████████| 93.8MB 57kB/s 
  Created wheel for spacy-Lookups-data: filename=spacy_lookups_data-0.3.2-py2.py3-none-any.whl size=93807573 sha256=4dc29fd5c3defd5e8929f3110ca5646650eb9f740017782fdc18ee5357074d09
  Stored in directory: /root/.cache/pip/wheels/5b/f4/d0/bf720a06127c95d9be2a81d197a3f1998ee5fc63410944e28f
Successfully built spacy-Lookups-data


In [15]:
!python -m spacy download en_core_web_sm 

     |████████████████████████████████| 12.1MB 529kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp36-none-any.whl size=12047109 sha256=471f1733d009b2c033f7e2cecf61ffa2f49ae452d2554e935e53e5b5d99991c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-uec_yk1m/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [16]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 50.8MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.3.1-cp36-none-any.whl size=50916643 sha256=405a3d556a1a633b623aefa9360d4025e2d4e7813015376f12c449ca9006a25f
  Stored in directory: /tmp/pip-ephem-wheel-cache-n0kn_a8_/wheels/6e/65/3a/34cdc26d4084d1d1f1e2ec9914964759ea17aa382c53a57d9f
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [17]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 782.7MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.3.1-cp36-none-any.whl size=782936125 sha256=3d37ebf583433e832766d0f34bc6acf786d37d7edec7dba417bf7603f7d4703f
  Stored in directory: /tmp/pip-ephem-wheel-cache-avg3mn03/wheels/ce/4d/1b/bc6cabb6df139c5f0318927be3ae9e51363fb44d6ea328d3f4
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [11]:
train_data = train_data[['Tweet','Stance']]

In [12]:
train_data['Stance'].value_counts()

0    1395
2     766
1     753
Name: Stance, dtype: int64

In [13]:
# Word count in the Tweet column 

train_data['Word_Count'] = train_data['Tweet'].apply(lambda x: len(str(x).split()))

In [14]:
## Character Counts in Tweet column

train_data['Character_Count'] = train_data['Tweet'].apply(lambda x: len(x))

In [15]:
train_data.head()

,Tweet,Stance,Word_Count,Character_Count
0,"@tedcruz And, #HandOverTheServer she wiped cle...",0,19,143
1,Hillary is our best choice if we truly want to...,1,18,105
2,@TheView I think our country is ready for a fe...,0,18,97
3,I just gave an unhealthy amount of my hard-ear...,0,21,140
4,@PortiaABoulger Thank you for adding me to you...,2,11,68


In [16]:
# Calculating the average word length [calculated using formula =  (Character_Count/Word_Count)]

train_data['Average_Word_length'] = train_data['Character_Count']/train_data['Word_Count']

In [17]:
train_data.head()

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length
0,"@tedcruz And, #HandOverTheServer she wiped cle...",0,19,143,7.526316
1,Hillary is our best choice if we truly want to...,1,18,105,5.833333
2,@TheView I think our country is ready for a fe...,0,18,97,5.388889
3,I just gave an unhealthy amount of my hard-ear...,0,21,140,6.666667
4,@PortiaABoulger Thank you for adding me to you...,2,11,68,6.181818


In [18]:
# Counting the stop words

train_data['Stop_words_count'] = train_data['Tweet'].apply(lambda x: len([t for t in x.split() if t in STOP_WORDS]))

In [19]:
train_data.head()

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count
0,"@tedcruz And, #HandOverTheServer she wiped cle...",0,19,143,7.526316,3
1,Hillary is our best choice if we truly want to...,1,18,105,5.833333,7
2,@TheView I think our country is ready for a fe...,0,18,97,5.388889,7
3,I just gave an unhealthy amount of my hard-ear...,0,21,140,6.666667,7
4,@PortiaABoulger Thank you for adding me to you...,2,11,68,6.181818,5


In [20]:
## Count HashTags -> # and mentions -> @

train_data['HashTage_Count'] = train_data['Tweet'].apply(lambda x : len([t for t in x.split() if t.startswith("#")])) 


train_data['Mention_Count'] = train_data['Tweet'].apply(lambda x : len([t for t in x.split() if t.startswith("@")])) 

In [21]:
train_data.head()

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count,HashTage_Count,Mention_Count
0,"@tedcruz And, #HandOverTheServer she wiped cle...",0,19,143,7.526316,3,3,1
1,Hillary is our best choice if we truly want to...,1,18,105,5.833333,7,1,0
2,@TheView I think our country is ready for a fe...,0,18,97,5.388889,7,0,1
3,I just gave an unhealthy amount of my hard-ear...,0,21,140,6.666667,7,1,0
4,@PortiaABoulger Thank you for adding me to you...,2,11,68,6.181818,5,0,1


In [22]:
## Checking the numeric digits present in the tweets

train_data['numeric_digit_Count'] = train_data['Tweet'].apply(lambda x : len([t for t in x.split() if t.isdigit()])) 

In [23]:
# Checking the count of Upper Case in the tweet (Because mostly people write in capital when they are Happy or Sad)

train_data['uppercase_Count'] = train_data['Tweet'].apply(lambda x : len([t for t in x.split() if t.isupper() and len(x)>3])) 

In [24]:
train_data.head()

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count,HashTage_Count,Mention_Count,numeric_digit_Count,uppercase_Count
0,"@tedcruz And, #HandOverTheServer she wiped cle...",0,19,143,7.526316,3,3,1,0,0
1,Hillary is our best choice if we truly want to...,1,18,105,5.833333,7,1,0,0,0
2,@TheView I think our country is ready for a fe...,0,18,97,5.388889,7,0,1,0,1
3,I just gave an unhealthy amount of my hard-ear...,0,21,140,6.666667,7,1,0,0,2
4,@PortiaABoulger Thank you for adding me to you...,2,11,68,6.181818,5,0,1,0,0


*Preprocessing and cleaning the data*

In [25]:
# Converting to lower case

train_data['Tweet'] = train_data['Tweet'].apply(lambda x: x.lower()) 

In [26]:
train_data['Tweet'].head(2)

0    @tedcruz and, #handovertheserver she wiped cle...
1    hillary is our best choice if we truly want to...
Name: Tweet, dtype: object

In [27]:
# Contraction to Expandion [Meaning converting words like "don't" to "do not"]

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i had",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it had",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"o'clock": "of the clock",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she had",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they had",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'll": "you will",
"you're": "you are",
"you've": "you have",
" n ":"and",
" u ": "you",
}

In [28]:
def con_to_ext(x):
  if type(x) is str:
    for key in contractions:
      value = contractions[key]
      x = x.replace(key, value)
    return x
  else:
    return x

In [29]:
%%time
train_data['Tweet'] = train_data['Tweet'].apply(lambda x: con_to_ext(x))

CPU times: user 87.3 ms, sys: 0 ns, total: 87.3 ms
Wall time: 93.9 ms


In [30]:
train_data

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count,HashTage_Count,Mention_Count,numeric_digit_Count,uppercase_Count
0,"@tedcruz and, #handovertheserver she wiped cle...",0,19,143,7.526316,3,3,1,0,0
1,hillary is our best choice if we truly want to...,1,18,105,5.833333,7,1,0,0,0
2,@theview i think our country is ready for a fe...,0,18,97,5.388889,7,0,1,0,1
3,i just gave an unhealthy amount of my hard-ear...,0,21,140,6.666667,7,1,0,0,2
4,@portiaaboulger thank you for adding me to you...,2,11,68,6.181818,5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2909,"there is a law protecting unborn eagles, but n...",0,25,156,6.240000,7,1,0,1,0
2910,i am 1 in 3... i have had an abortion #abortio...,0,16,108,6.750000,6,3,0,1,2
2911,how dare you say my sexual preference is a cho...,0,23,142,6.173913,10,2,0,0,0
2912,"equal rights for those 'born that way', no rig...",0,21,137,6.523810,9,4,0,0,0


In [31]:
## Count and remove emails from the tweet

import re

train_data['email'] = train_data['Tweet'].apply(lambda x: re.findall(r'([-a-zA-Z0-9._-]+@[-a-zA-Z0-9._-]+\.[-a-zA-Z0-9_-]+)',x))


In [32]:
# Counting the number of email present in the dataset 

train_data['email_count'] = train_data['email'].apply(lambda x: len(x))

In [33]:
# Checking if the emails are greater that 1

train_data[train_data['email_count']>0]

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count,HashTage_Count,Mention_Count,numeric_digit_Count,uppercase_Count,email,email_count
676,(2/2) 300 words and be sent to climateconferen...,1,17,142,8.352941,5,3,0,1,0,[climateconference2015@gmail.com],1
2691,worldwide r.e. agents! - support a prolife gro...,0,16,129,8.062500,3,1,0,0,2,[proliferealestate@yahoo.com],1


In [34]:
## Removing the emails from the Tweet column.

train_data['Tweet'] = train_data['Tweet'].apply(lambda x: re.sub(r'([-a-zA-Z0-9._-]+@[-a-zA-Z0-9._-]+\.[-a-zA-Z0-9_-]+)','',x))

In [35]:
# Counting the url link in the tweet column

train_data['url_count'] = train_data['Tweet'].apply(lambda x: len(re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',x)))

In [36]:
# Checking the url in the dataset

train_data[train_data['url_count']>0]

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count,HashTage_Count,Mention_Count,numeric_digit_Count,uppercase_Count,email,email_count,url_count


In [37]:
### Removing the RT [RT -> Retweet]

train_data['Tweet'] = train_data['Tweet'].apply(lambda x : re.sub('RT', "", x))

In [38]:
train_data['Tweet']

0       @tedcruz and, #handovertheserver she wiped cle...
1       hillary is our best choice if we truly want to...
2       @theview i think our country is ready for a fe...
3       i just gave an unhealthy amount of my hard-ear...
4       @portiaaboulger thank you for adding me to you...
                              ...                        
2909    there is a law protecting unborn eagles, but n...
2910    i am 1 in 3... i have had an abortion #abortio...
2911    how dare you say my sexual preference is a cho...
2912    equal rights for those 'born that way', no rig...
2913    #potus seals his legacy w/ 1/2 doz wins. the #...
Name: Tweet, Length: 2914, dtype: object

In [39]:
### Removing Special characters and punctuations

train_data['Tweet'] = train_data['Tweet'].apply(lambda x: re.sub('[^A-Za-z0-9]+',' ',x))

In [40]:
### Remove the multiple(extra) spaces

train_data['Tweet'] = train_data['Tweet'].apply(lambda x: " ".join(x.split()))

In [41]:
### Remove the HTML tags [e.g. <h1> hello <h1>]

from bs4 import BeautifulSoup

train_data['Tweet'] = train_data['Tweet'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [42]:
### Remove the Accented characters (e.g. àÿüûâ)
import unicodedata

def remove_accent(x):
  x = unicodedata.normalize('NFKD', x).encode('ascii','ignore').decode('utf-8','ignore')
  return x

## Applying on tweets
train_data['Tweet'] = train_data['Tweet'].apply(lambda x: remove_accent(x)) 

In [43]:
### Removing the stop words

train_data['Tweet'] = train_data['Tweet'].apply(lambda x: " ".join([t for t in x.split() if t not in STOP_WORDS]))

In [44]:
 ## Removed the stop words
 
 train_data['Tweet']

0       tedcruz handovertheserver wiped clean 30k dele...
1       hillary best choice truly want continue progre...
2       theview think country ready female pres hillar...
3       gave unhealthy hard earned money away big gov ...
4        portiaaboulger thank adding list hillary clinton
                              ...                        
2909    law protecting unborn eagles humans uh idk not...
2910    1 3 abortion abortionondemand menstruationmatt...
2911    dare sexual preference choice dare dismemberin...
2912    equal rights born way rights born liberallogic...
2913    potus seals legacy w 1 2 doz wins gop agenda r...
Name: Tweet, Length: 2914, dtype: object

In [45]:
## Converting words into its Root/Base form (Lemmatization)

import nltk
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

train_data['Tweet'] = train_data.Tweet.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
train_data['Tweet']

0       [tedcruz, handovertheserver, wiped, clean, 30k...
1       [hillary, best, choice, truly, want, continue,...
2       [theview, think, country, ready, female, pres,...
3       [gave, unhealthy, hard, earned, money, away, b...
4       [portiaaboulger, thank, adding, list, hillary,...
                              ...                        
2909    [law, protecting, unborn, eagle, human, uh, id...
2910    [1, 3, abortion, abortionondemand, menstruatio...
2911    [dare, sexual, preference, choice, dare, disme...
2912    [equal, right, born, way, right, born, liberal...
2913    [potus, seal, legacy, w, 1, 2, doz, win, gop, ...
Name: Tweet, Length: 2914, dtype: object

In [47]:
## convert the list to string

train_data['Tweet'] = train_data.Tweet.apply(' '.join)

In [274]:
train_data['Tweet']

0       tedcruz handovertheserver wiped clean 30k dele...
1       hillary best choice truly want continue progre...
2       theview think country ready female pres hillar...
3       gave unhealthy hard earned money away big gov ...
4        portiaaboulger thank adding list hillary clinton
                              ...                        
2909    law protecting unborn eagle human uh idk notic...
2910    1 3 abortion abortionondemand menstruationmatt...
2911    dare sexual preference choice dare dismemberin...
2912    equal right born way right born liberallogic l...
2913    potus seal legacy w 1 2 doz win gop agenda rep...
Name: Tweet, Length: 2914, dtype: object

In [48]:
### Common word removal. [[DOUBT]] Do we need to remove this frequently(top 20 ) occuring words

text = " ".join(train_data['Tweet'])
text = text.split()
freq_com = pd.Series(text).value_counts()

In [49]:
### Common word removal. [[DOUBT]] Do we need to remove this frequently occuring words

top_f_20 = freq_com[:20]

rare_f_20 = freq_com[-20:]

In [280]:
rare_f_20

antibibles           1
dnt                  1
doodisgirl           1
togodbetheglory      1
popupincubator       1
scenario             1
sunscreen            1
lordoflords          1
estab                1
rodeo                1
ndnstyl              1
egregious            1
tasty                1
accountable          1
inspiredbyhillary    1
commodity            1
circumstance         1
inching              1
basing               1
ancient              1
dtype: int64

In [ ]:
### Common word removal. [[DOUBT]] Do we need to remove this frequently occuring words

#### train_data['Tweet'] = train_data['Tweet'].apply(lambda x: " ".join([t for t in x.split() if t not in top_f_20]))

In [ ]:
## Removing least frequently occuring words [[DOUBT]] Do we need to remove this less frequently occuring words

#### train_data['Tweet'] = train_data['Tweet'].apply(lambda x: " ".join([t for t in x.split() if t not in rare_f_20 ]))

In [50]:
## checking displaying rarely occuring words for whole dataset

freq_com[freq_com == 1]

chels              1
ecstatic           1
formidable         1
profeminist        1
oppose             1
                  ..
cocaine            1
autobailout        1
thegoodgodabove    1
agonist            1
blackkids          1
Length: 5390, dtype: int64

In [282]:
!pip install wordcloud

In [51]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
words = ''.join(text[:3000])

In [54]:
### Code to show the images using wordcloud.

# wordcloud = WordCloud(width=100, height=400).generate(words)
# plt.imshow(words)
# plt.axis('off')
# plt.show()

In [56]:
####  installing packages and corpus for spelling correction

#!pip install -U textblob
#!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
### Spelling Correction

from textblob import TextBlob
 
train_data['Tweet'] = train_data['Tweet'].apply(lambda x: TextBlob(x).correct()))

In [ ]:
### Lemmatization using spacy

# def tok_lemma(x):
#   doc = nlp()
#   for token in doc:
#     token.lemma_

In [ ]:
### Lemmatization using Spacy
# train_data['Tweet'] = train_data['Tweet'].apply(lambda x: tok_lemma(x)) 

In [57]:
train_data

,Tweet,Stance,Word_Count,Character_Count,Average_Word_length,Stop_words_count,HashTage_Count,Mention_Count,numeric_digit_Count,uppercase_Count,email,email_count,url_count
0,tedcruz handovertheserver wiped clean 30k dele...,0,19,143,7.526316,3,3,1,0,0,[],0,0
1,hillary best choice truly want continue progre...,1,18,105,5.833333,7,1,0,0,0,[],0,0
2,theview think country ready female pres hillar...,0,18,97,5.388889,7,0,1,0,1,[],0,0
3,gave unhealthy hard earned money away big gov ...,0,21,140,6.666667,7,1,0,0,2,[],0,0
4,portiaaboulger thank adding list hillary clinton,2,11,68,6.181818,5,0,1,0,0,[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2909,law protecting unborn eagle human uh idk notic...,0,25,156,6.240000,7,1,0,1,0,[],0,0
2910,1 3 abortion abortionondemand menstruationmatt...,0,16,108,6.750000,6,3,0,1,2,[],0,0
2911,dare sexual preference choice dare dismemberin...,0,23,142,6.173913,10,2,0,0,0,[],0,0
2912,equal right born way right born liberallogic l...,0,21,137,6.523810,9,4,0,0,0,[],0,0


In [59]:
# Dividing the dataset into train and test set

X = train_data['Tweet']

In [66]:
### Preprocessed words

preprocessed_words = []
for i in X:
  preprocessed_words.append(i)

In [67]:
print(preprocessed_words)

['tedcruz handovertheserver wiped clean 30k deleted email explains dereliction duty lie benghazi etc tcot hillary clinton', 'hillary best choice truly want continue progressive nation ohio hillary clinton', 'theview think country ready female pres hillary hillary clinton', 'gave unhealthy hard earned money away big gov t untrustworthy irs whyimnotvotingforhillary hillary clinton', 'portiaaboulger thank adding list hillary clinton', 'hillary win s hoping dems offer real candidate like warren warren2016 hillary clinton', 'respect law respect law yes needed desperately baltimoreriots hillary clinton', 'want appointed ambassador post hillary clinton', 'stophillary2016 hillaryclinton woman integrity honesty vote woman president hillary clinton', 'hillaryclinton end lawless clintonfoundation jail butcher benghazi arrest rapist billclinton hillaryclinton hillary clinton', 'use brain hillary white house clinton2016 hillary clinton', 'hillaryclinton hillary pandering logo clintonfoundationscand

In [60]:
y = train_data['Stance']

In [61]:
X.shape

(2914,)

In [62]:
y.shape

(2914,)

In [68]:
### Defining the vocabulary size

voc_size = 10000  ## Also have to Check with size 5000 

In [69]:
## One Hot representation 

onehot_repr = [one_hot(word,voc_size) for word in preprocessed_words]

## Displaying the one-hot representated values

# onehot_repr

In [70]:
## Embedding Representation

sentence_length = 30
embedded_doc = pad_sequences(onehot_repr, padding='pre', maxlen=sentence_length)

In [71]:
### BaseLine Model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 456,703
Trainable params: 456,703
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

import numpy as np
X_final=np.array(embedded_doc)
y_final=np.array(y_binary)

In [74]:
### Dividing into train and test set.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [75]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
31/31 [==============================] - 2s 62ms/step - loss: 1.0555 - accuracy: 0.4764 - val_loss: 1.0361 - val_accuracy: 0.4688
Epoch 2/10
31/31 [==============================] - 1s 45ms/step - loss: 0.9402 - accuracy: 0.5394 - val_loss: 0.9348 - val_accuracy: 0.5437
Epoch 3/10
31/31 [==============================] - 1s 45ms/step - loss: 0.6690 - accuracy: 0.6931 - val_loss: 0.9874 - val_accuracy: 0.5405
Epoch 4/10
31/31 [==============================] - 1s 46ms/step - loss: 0.4983 - accuracy: 0.7725 - val_loss: 1.2806 - val_accuracy: 0.5260
Epoch 5/10
31/31 [==============================] - 1s 45ms/step - loss: 0.3727 - accuracy: 0.8361 - val_loss: 1.6652 - val_accuracy: 0.4543
Epoch 6/10
31/31 [==============================] - 1s 45ms/step - loss: 0.4068 - accuracy: 0.8386 - val_loss: 1.1792 - val_accuracy: 0.5478
Epoch 7/10
31/31 [==============================] - 1s 45ms/step - loss: 0.1721 - accuracy: 0.9498 - val_loss: 1.6070 - val_accuracy: 0.5343
Epoch 8/10
31

In [76]:
# Converting the one-hot encoded to single digit

import numpy as np
rounded_labels=np.argmax(y_test, axis=1)
rounded_labels[155]

1

In [78]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [79]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(rounded_labels, y_pred)

array([[[354, 157],
        [151, 300]],

       [[567, 127],
        [149, 119]],

       [[575, 144],
        [128, 115]]])

In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(rounded_labels,y_pred)

0.5550935550935551